In [ ]:
# 確認各位chrom版本 chrome://settings/help
# 依照自己的版本去下載driver https://chromedriver.chromium.org/downloads
# 安裝selenium 工具包 (pip install selenium), 模擬自動化爬蟲流程
# 如果要對driver 有一些視窗上的操作, 參考 https://www.itread01.com/content/1544483715.html
!pip install selenium

In [ ]:
# 嘗試一下用自動化工具開啟chrome 
# 1. 重點: 之前做的盒子的架構

In [ ]:
from selenium import webdriver #啟動控制我們的driver 
# 如果'chromedriver.exe' servr 有問題的話,跑一下以下:
    # from selenium.webdriver.chrome.service import Service
    # import sys
    # s = Service("chromedriver.exe")
    # driver = webdriver.Chrome(service=s)
url = 'https://shopee.tw/'
driver = webdriver.Chrome('chromedriver.exe') #開啟chrome
driver.get(url)

#### 使用seleinum 爬取蝦皮
###### 操作思路: 打開網頁 -> 全螢幕 -> 找搜尋欄位 -> 下關鍵字搜尋 ->滾動滑鼠滾輪瀏覽畫面 -> 得到新的盒子資訊....

In [22]:
from selenium import webdriver #啟動控制我們的driver 
from selenium.webdriver.common.keys import Keys #模擬使用鍵盤
from selenium.webdriver.chrome.options import Options # 可以對網頁開啟時,做一些篩選,例如關閉開啟網頁的自動翻譯/詢問
import time
#==================Explicit Waits=====(等待driver運作,直到某個特徵出現為止)========================
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

chrom_driver_path = 'chromedriver.exe'
url = 'https://shopee.tw/'
item = '包包'
opt = Options()
opt.add_argument('--disable-notifications') #禁止彈出詢問視窗

driver = webdriver.Chrome(chrom_driver_path, chrome_options = opt)
driver.get(url) # 得到url資訊

#放大全螢幕
driver.maximize_window()

# 做一個彈性的等待(因為driver在處理時要時間,不想用sleep去固定時間等待)
# 最多等10秒, 直到畫面有'stardust-popover__target' 出現後才會繼續走程式碼
WebDriverWait(driver,10).until(EC.presence_of_element_located( (By.CLASS_NAME,'stardust-popover__target') ) )
search = driver.find_elements_by_class_name('shopee-searchbar-input__input') #類似bs4 find_all
# print('!!!!',search)
keyin_bar = search[0]
keyin_bar.send_keys(item)
keyin_bar.send_keys(Keys.ENTER)
# 發現包包的最大單一大盒子的class : col-xs-2-4 shopee-search-item-result__item 
WebDriverWait(driver,10).until(EC.presence_of_element_located( (By.CLASS_NAME,'col-xs-2-4') ) )
print('已經搜尋完畢..')

# 玩一下滾動輪子, 因為一開始網頁的盒子,注意一下商品的盒子,當滑鼠沒有滾的時候,雖然有框架,但是沒有內容資訊,一定要
# 滾動一下滑鼠,才有盒子資訊注入
# pix = 500
# for i in range(2):
#     js = f'window.scrollBy(0,{pix})'
#     driver.execute_script(js)
#     print('滑鼠滾動一次~')
#     time.sleep(1)

# 把滾輪注入的定義寫好~~
# prods = driver.find_elements_by_class_name('tWpFe2') # 找一下產品大盒子(沒有flex的)
# img_box = prods[0].find_elements_by_class_name('_7DTxhh') #找一下img 的盒子
# img_link = img_box[0].get_attribute('src') #找一下圖片連結
# title = img_box[0].get_attribute('alt')

def mouse_roll_and_crawl( title_list , driver , pix =500 , run_time = 10):
    global imgs
    #偷偷寫一下,輸入(添加名稱的籃子, driver , 每次向下滑多少像素, 總共滾幾次)
    for each_move in range(run_time):
        js = f'window.scrollBy(0,{pix})'
        driver.execute_script(js)
        print('滑鼠滾動一次~')
        time.sleep(1)
        prods = driver.find_elements_by_class_name('_7DTxhh')  #找一下img 的盒子 /裡面有所有資訊
        for img_box in prods:
            img_link = img_box.get_attribute('src') #找一下圖片連結
            title = img_box.get_attribute('alt')
            print(img_link)
            print(title)
            if title not in title_list: 
                #每次滾動 會有重複的內容, 做一個判斷,如果有重複的文字在已經append的籃子, 那就不要走這個判斷式
                title_list.append(title)
        time.sleep(1)
        print('每次滾完滑鼠,看一下籃子裡面的數量:',len(title_list))
    print('已經滾完所有頁面了,為了確保畫面有頁碼,所以滾輪網上滾一點點')
    js = f'window.scrollBy(0,-10)'
    driver.execute_script(js)    

# 調用函式, 每滾動500個pixcel 總共滾8次   
title_list = []
mouse_roll_and_crawl( title_list , driver , pix =500 , run_time = 8)
# title_list
# 去找buttom的盒子,並且右鍵,查看他的盒子的絕對路徑, 發現不同路徑只是盒子的路徑最後一個欄位不一樣,且page剛好是想要的頁數+1
for page in range(2,4):
    #/html/body/div[1]/div/div[2]/div/div/div[2]/div[2]/div[3]/div/button[3]  第二頁按鈕
    #/html/body/div[1]/div/div[2]/div/div/div[2]/div[2]/div[3]/div/button[4]  第三頁按鈕
    buttom_xpath = f'/html/body/div[1]/div/div[2]/div/div/div[2]/div[2]/div[3]/div/button[{page+1}]'
    driver.find_element_by_xpath(buttom_xpath).click() #用畫面的絕對位置,去找按鈕,並且點擊
    #進入下一頁 因為跳頁,會等一下下,所以確認下一頁會有的盒子是否出現
    WebDriverWait(driver,10).until(EC.presence_of_element_located( (By.CLASS_NAME,'_7DTxhh') ) )
    #成功跳轉到下一頁
    mouse_roll_and_crawl( title_list , driver , pix =500 , run_time = 8)
    print(f'==========第{page}頁完成爬取===========')
driver.quit()

C:\Users\User\Anaconda3\envs\tf27\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  app.launch_new_instance()
C:\Users\User\Anaconda3\envs\tf27\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: use options instead of chrome_options
  app.launch_new_instance()
C:\Users\User\Anaconda3\envs\tf27\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead


已經搜尋完畢..
滑鼠滾動一次~


C:\Users\User\Anaconda3\envs\tf27\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead


https://cf.shopee.tw/file/5b0db75a56b9608e8d5aed6a0deb9afe_tn
100%正品 美國代購 女生包包 新款小號水桶包 經典老花斜背包 女用手提包 精品斜跨肩背包 2312 附購證
https://cf.shopee.tw/file/08533780b439754ed982780099d0f08b_tn
MONETSHOP 貓貓女孩 女包腋下包法國小眾法棍包 鏈條手提單肩 2022新款潮斜跨包 側背包 送女朋友禮物
https://cf.shopee.tw/file/720ee0e8e05ad9ca5bd18d6aa5e57b66_tn
莎莎愛購🔥附購証【免運】包包新款菱格流蘇相機包 鏈條包 單肩包 斜挎包 斜背包 女生包包 側背包 側包 相機包
https://cf.shopee.tw/file/1b10cc78c3624d894fdd0be4cd65e4ea_tn
#加拿大代購# 100%正品有發票，保修卡！潼媽 COACH/蔻馳 男士長款錢包禮盒套裝 F37943 長夾
https://cf.shopee.tw/file/56a1c7578d856d870c1899b5144318d4_tn
【速出現貨】POLENE 腋下包 褶皺 托特包 小眾設計 新款潮 時尚 poleno女包
https://cf.shopee.tw/file/dc683a85ac188375d0ab1209f5b41c97_tn
👜cherish‘ 包包館|小包包 包包女 斜挎百搭ins學生圓韓版印花大容量寬肩提包
https://cf.shopee.tw/file/c8e936cbd8a105ea1187621f000d7437_tn
包包女 【現貨】 原創甜美花朵小包包時尚包包女2021新款學生黨可愛百搭仙女斜背包
https://cf.shopee.tw/file/cf3eaa51feaa716cd284d29ed74249f7_tn
𝐔𝐌𝐄𝐎𝐖 𝐒𝐓𝐔𝐃𝐈𝐎 時光裡的溫柔🌹菱格帆布小方包（4colors）斜背包
https://cf.shopee.tw/file/7da036459715bf8e427ea1dc6ccf4640_tn
包包🌷法國小眾款格菱包｜闆娘自留款
https://cf.shopee.tw/file/3a6

https://cf.shopee.tw/file/13ce7bfdace00b341713f8f3238ebc82_tn
MM6  小號經典三角包 Japanese tote Bag Small Truangle
https://cf.shopee.tw/file/fb30caf32e87d8128e6455a36bf7f333_tn
現貨速發 手提托特包  手提女包  帆布托特包  防水托特包 男包新款尼龍手提包男帆布商務包大容量休閒豎款潮男士手提包布
https://cf.shopee.tw/file/017f68858a1c89fae728db9d07c5f8ae_tn
優質~韓版小眾手提包包女新款時尚褶皺腋下包百搭斜挎包單肩包
https://cf.shopee.tw/file/71de03774dd441e64e5c43dd655540da_tn
單肩包 多層小方包 包包 小方包 韓版包包 斜挎包 女包 包包女 側背包 肩背包 斜背包 小廢包 小包包 斜跨包
https://cf.shopee.tw/file/2effc56afa8c38fd6f66b36e037b025b_tn
👑網絡熱銷款 大容量肩背手提兩用包 包包 新款 韓版 女包 百搭 手提包 單肩 側背包 大包包 單肩包 斜背包
https://cf.shopee.tw/file/1f7be3c453304fe5763bc9bc650a815b_tn
【Sanm·三木】斜挎包 側背包 韓版ins學院風軟妹餃子包華夫格燈芯絨單肩包 日系原宿古著感少女斜挎包
https://cf.shopee.tw/file/52adf1339da82eab996708f41310390e_tn
【Hermosa】 韓版馬鞍包 小清新包包 純色單肩包 斜肩包 百搭時尚側背包 休閒小方包 韓國ins 斜背包 側背包
https://cf.shopee.tw/file/9b2a460f5aca337bd9c805b6a34b2cea_tn
肩背包 撞色斜背小方包 19069 鏈帶包 斜背包 小方包 單肩包 台灣現貨
https://cf.shopee.tw/file/496d98bda73644bcf2afe22a65a34d44_tn
ATTENTION-新色回歸!att made!小貝比包
https://cf.

...

https://cf.shopee.tw/file/e7300ade10a5801564b5658a700d883b_tn
N76✨現貨韓系/雲朵包/小包/ins/小廢包/肩背包/斜背包/手提包/小方包/褶皺包/皺皺包/休閒包/小包包/韓妞必備
https://cf.shopee.tw/file/94cfd9c2d1563820c9b8b8f155c664fb_tn
黃金歲月般的馬鞍小包 21880009
https://cf.shopee.tw/file/c7c6cbaa2e6f0ae5a3142edf8f0980bc_tn
MICHAEL KORS 後背包
https://cf.shopee.tw/file/9846018936a36d366f2f704d0efe3526_tn
元元🍦高級設計感包包 小ck新款小香風菱格鏈條包包女2021潮百搭斜挎包Chic韓版小包包
https://cf.shopee.tw/file/84c7f443de661108517dc79e76054b64_tn
🇰🇷【快速出貨】韓國KKOBA NERDY 22新款 愛心包 韓國代購 斜背包 手提包 NERDY愛心包 韓國包
https://cf.shopee.tw/file/fb0d3ba43ac21ef3965d51600695c865_tn
🔥買一送一🔥內有實拍影片💯Michael kors Mk防刮兩用斜背包 手提包 劍橋包
https://cf.shopee.tw/file/68172586f6201aba289cd65eb234b3fe_tn
現貨滿額免運 小香風時尚手提袋帆布字母四方大容量便攜手拎媽咪包上班購物袋
https://cf.shopee.tw/file/1fbb08d625cd1c2caa801da0c57f39b6_tn
【暖暖家】純色毛絨包可愛女包毛毛女2021秋冬新款時尚洋氣可愛單肩腋下包毛絨百搭手拎小包
https://cf.shopee.tw/file/1294360516491c9e21e5df039e73c6a1_tn
ISSEY MIYAKE 三宅一生BAOBAO霧面幾何方格斜背包(湖綠色)
https://cf.shopee.tw/file/b5cdba1be0c57451889fa3f40f10e408_tn
精緻LOGO水桶包 包包 單肩

https://cf.shopee.tw/file/203840eb4abac8a4c45049af3f82ae8e_tn
𝙋𝙞𝙣𝙠 ♡個性黑色鏈條單肩包/側背包
https://cf.shopee.tw/file/bcde8e40533531aac37860eddf2874eb_tn
《MR.JK》韓國 FIND KAPOOR 翻蓋包 MARTY 12/18/26 李聖經 水晶 李孝利 小廢包 FK
https://cf.shopee.tw/file/e023f8c8f416b5f273040eebc80b131a_tn
8樂雜貨舖 韓版小方包 小方包 氣質斜背小包 斜背包 單肩包 女小包 斜跨包 女生包包 撤背包側包側背小方包小包包女包
https://cf.shopee.tw/file/b279d796a0b07c2685456e7ed1cc5d9a_tn
[現貨] 金釦皮面氣質小包包*4色 闆娘推薦款!! / 包
https://cf.shopee.tw/file/c599466f3e22be608aac6357c04c0861_tn
台灣現貨🔥法式水桶包🎐野餐露營 便當袋側背包 斜背包 單肩 女孩 嬰幼 禮物 ipad 約會 秋冬灰 黃生日 黑 粉卡其
https://cf.shopee.tw/file/b5c5da2cce9d5343ad0f7d6e7130742a_tn
#二手包 斷捨離 Michael Kors WOC包（不議價可直接下單）
https://cf.shopee.tw/file/44e42ae7e7564ff25dfe2b4c7e9145e1_tn
現貨 5639-菱格單肩斜跨包 小香風 單肩包 斜跨包 手機包 小包包 小方包  韓版 ins 背包 韓國 東大
https://cf.shopee.tw/file/0605abf7ff45bdd4affac8b359fd9fb7_tn
【白現貨】Tintin Korea。正韓 Toae 真皮 義大利牛皮可調節肩背包 Pillow Bag
https://cf.shopee.tw/file/7a5dfa4fe98542a00824d5a9aa0e4928_tn
貓咪包 貓貓包 牙獵犬抱枕 仿真貓咪公仔背包  雙肩包 大容量背包
https://cf.shopee.tw/file/58fd1d

https://cf.shopee.tw/file/df1282f5befddfcffe2183af07cd6fd9_tn
（送紙袋）現貨🇹🇼小CK air pods耳機套/耳機包
https://cf.shopee.tw/file/69ed6af3bbb13e1e8ca74c50d05cbc21_tn
【台灣出貨】小眾手提包 韓國女包包 2022夏新款尼龍百搭mini手提小包包女 通勤手拎包 斜挎包 約會逛街女包包
https://cf.shopee.tw/file/1668b3e82d8e427760a09416b6104b11_tn
【全館49元專賣】狗狗斜挎包 帆布斜挎包 帆布小包包 斜挎包ins 少女小包包 狗狗包 日系小包包 日系帆布
https://cf.shopee.tw/file/9642039663e40eedb2a6f96376f9df71_tn
現貨 泰國🇹🇭 RUST BRAND小眾設計師品牌 簡約質感托特包
https://cf.shopee.tw/file/4310a118727bf72d8e61f4980f12680e_tn
超軟牛角包 月牙包 水餃包 包包 包包女 餃子包 馬鞍包 單肩包 側背包 流浪包 斜背包 半月包 腋下包 女包 B36
https://cf.shopee.tw/file/4bac502259864143ae742394ac43565e_tn
【青春派】夏季時尚小包包2020新款潮亮片鏈條單肩斜挎包女包百搭ins小方包
https://cf.shopee.tw/file/69f0ce4b90195832e7e3cd6daa9ef788_tn
MK二手斜背腰包附防塵袋
https://cf.shopee.tw/file/e22b2896884ec8dfbc2aaf8edca126d0_tn
❗️現貨❗️可愛少女吐司包 煎蛋小包🍞 時尚獨家斜肩小包🎁
https://cf.shopee.tw/file/cbc52997d024b6daeb5387587e8aa4be_tn
今年流行小包包女夏小眾新款潮百搭ins鏈條包爆款單肩斜挎包时尚包包女 韩系包包
https://cf.shopee.tw/file/8ffe586fc72a74349690f6e1518b27db_tn
🌸預購+現貨🌸2021韓國🇰🇷最新百搭斜背半

每次滾完滑鼠,看一下籃子裡面的數量: 175
滑鼠滾動一次~
https://cf.shopee.tw/file/ce5434f17694e364a009dab3e2cc8d1d_tn
促銷款 日式和風帆布包可愛兔耳朵便當包上班帶飯學生午餐飯包
https://cf.shopee.tw/file/93316fc798bfb5f04cdee8918d12b1f3_tn
Yanyan 小艷精品 美國代購 女生相機包 黑色荔枝紋 側背包 雙層拉鍊單肩包 大容量包包 素面斜挎包 女生包包 禮物
https://cf.shopee.tw/file/21a37350d5e04d1b20d91977e1846321_tn
✨詩詩美國🇺🇸代購 100%正品附購證 COACH 蔻馳 經典LOGO拉鏈皮夾 長夾 手拿包 女士錢包F52859
https://cf.shopee.tw/file/b533b4ae5995a77e7dc78e46be9cd4d5_tn
默默代購正品 新款 52372 防刮荔枝紋拉鏈 女生皮夾 長夾 皮革手拿包 零錢包 多卡位 鈔票夾
https://cf.shopee.tw/file/45a3dfd56f5503fef602a7eb252a2b35_tn
明星同款韓版時尚鑲鑽小方包 新款女斜挎包 楊冪同款 李菲兒同款 斜挎包 手提包 鏈條小方包 單肩包 手腕包 鑲鑽 包包
https://cf.shopee.tw/file/e2ec7a4ae5b86dd36ef22a40f6ac4878_tn
#現貨‼️迷你豆腐小包ꪔ̤̫‬ꪔ̤̱ꪔ̤̮ꪔ̤̥
https://cf.shopee.tw/file/ad0a961e78940361ff2535f1eb806032_tn
🔥熱銷商品🔥22韓版時尚潮流斜挎包 休閒手提包袋鼠小包
https://cf.shopee.tw/file/d128dbc09ab4283d94dd94fd79287f57_tn
❤️滿199免運💛網紅小包包女包2022夏天新款單肩手提斜挎百搭ins時尚鏈條小方包百搭包包女ins/
https://cf.shopee.tw/file/f37d01f8e0b59519a4c756734302b3a8_tn
𝐔𝐌𝐄𝐎𝐖 𝐒𝐓𝐔𝐃𝐈𝐎 春天來臨前 翻蓋大容量單肩包（3colors）
https://cf

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=105.0.5195.127)
Stacktrace:
Backtrace:
	Ordinal0 [0x00A8DF13+2219795]
	Ordinal0 [0x00A22841+1779777]
	Ordinal0 [0x0093423D+803389]
	Ordinal0 [0x00936D04+814340]
	Ordinal0 [0x00936BC2+814018]
	Ordinal0 [0x0093755F+816479]
	Ordinal0 [0x0098FC1B+1178651]
	Ordinal0 [0x0097E7FC+1107964]
	Ordinal0 [0x0098F192+1175954]
	Ordinal0 [0x0097E616+1107478]
	Ordinal0 [0x00957F89+950153]
	Ordinal0 [0x00958F56+954198]
	GetHandleVerifier [0x00D82CB2+3040210]
	GetHandleVerifier [0x00D72BB4+2974420]
	GetHandleVerifier [0x00B26A0A+565546]
	GetHandleVerifier [0x00B25680+560544]
	Ordinal0 [0x00A29A5C+1808988]
	Ordinal0 [0x00A2E3A8+1827752]
	Ordinal0 [0x00A2E495+1827989]
	Ordinal0 [0x00A380A4+1867940]
	BaseThreadInitThunk [0x7712FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77E47A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77E47A6E+238]


61